In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Data Manipulation (ETL)

In [ ]:
import pandas as pd
import os

# Define the folder paths
folder_paths = ['/content/drive/MyDrive/Docs - gerais/Finanças/Filipe', '/content/drive/MyDrive/Docs - gerais/Finanças/Giovanna']

# Create an empty list to store the dataframes
dfs = []

# Loop through each folder path
for folder_path in folder_paths:
    # List all files in the folder
    files = os.listdir(folder_path)
    print(files)

    # Filter out only Excel files
    excel_files = [file for file in files if file.endswith('.xlsx')]
    print(excel_files)
    # Loop through each Excel file in the folder
    for file in excel_files:
        # Construct the full file path
        file_path = os.path.join(folder_path, file)
        # Read Excel file into a dataframe
        df = pd.read_excel(file_path)
        # Add a new column indicating the source folder
        df['Source_Folder'] = os.path.basename(folder_path)
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(dfs, ignore_index=True)

# Remove duplicates
combined_df = combined_df.drop_duplicates()
combined_df['Period'] = pd.to_datetime(combined_df['Period']).dt.date
combined_df['period_yy_mm'] = pd.to_datetime(combined_df['Period']).dt.strftime('%m-%Y')
combined_df['EUR_pos_neg'] = combined_df['EUR']
combined_df['EUR_salary'] = 0
combined_df['EUR_no_salary'] = 0
# Apply the condition to multiply 'EUR_no_salary' by -1 where 'Category' contains 'Salary'
combined_df.loc[combined_df['Income/Expense'].str.contains('Exp.'), 'EUR_pos_neg'] *= -1

combined_df.loc[combined_df['Income/Expense'].str.contains('Exp.'), 'EUR_no_salary'] = combined_df['EUR']

combined_df.loc[combined_df['Income/Expense'].str.contains('Income'), 'EUR_salary'] = combined_df['EUR']

combined_df.head(50)

Save DF as Google Sheets file

In [ ]:
from google.colab import auth
from google.auth import default

try:
  import gspread
except ModuleNotFoundError:
  if 'google.colab' in str(get_ipython()):
    %pip install gspread
  import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
worksheet = gc.open("Controle de Despesas dos Bolinhos")
sheet1 = worksheet.sheet1

combined_df['Period'] = combined_df['Period'].astype(str)
sheet1.batch_clear(['A1:Z50000'])
sheet1.update([combined_df.columns.values.tolist()] + combined_df.fillna(-1).values.tolist())

{'spreadsheetId': '1fmwiQIQd8nCrCghd5n8cpll8opJGiMtWz9Hs--J3z4w',
 'updatedRange': 'raw!A1:P598',
 'updatedRows': 598,
 'updatedColumns': 16,
 'updatedCells': 9568}